# Modelling

## Import required moduel

In [1]:
import numpy as np
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

2023-03-29 14:07:57.298149: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Instructions for updating:
non-resource variables are not supported in the long term


## 1- Prepare the train and test data 

In [2]:
train_data = np.load('/home/alisina/Desktop/8th Semester/condidate #2/Code/Sign Language Recognition/Result/Preprocessing/Numpy Array Data & labels/train_data.npy', allow_pickle = True)
test_data = np.load("/home/alisina/Desktop/8th Semester/condidate #2/Code/Sign Language Recognition/Result/Preprocessing/Numpy Array Data & labels/test_data.npy", allow_pickle = True)

In [3]:
train_data.ndim # dimensions of train data
test_data.ndim # dimensions of test data

2

In [4]:
# Train Category
X_train_data = train_data[::, 0]
Y_train_data = train_data[::, 1]

# Test Category
x_test_data = test_data[::, 0]
y_test_data = test_data[::, 1]

In [ ]:
print("Length of X Train: ", len(X_train_data))
print("Length of Y Train: ", len(Y_train_data))
print()
print("Length of x Test: ", len(x_test_data))
print("Length of y Test: ", len(y_test_data))

## Scalling the Data
* If you don't need to scale data then you can ignore this part and go through to the next part ...
* Alert! If you do this part then please ignore the next part and go through to the <b>Create the Model Part</b>.

In [ ]:
X_train_data = X_train_data / 255

In [ ]:
x_test_data = x_test_data / 255

In [ ]:
X_train_data = np.array([item for item in X_train_data]).reshape(-1, 96, 96, 1)

In [ ]:
x_train_data = np.array([item for item in x_train_data]).reshape(-1, 96, 96, 1)

In [ ]:
# Y train category
targets = np.array(Y_train_data).reshape(-1)
print(X_train_data[0])
print(targets)
y_train1 = np.eye(32)[targets]

# y test category
targets = np.array(y_test_data).reshape(-1)
print(x_test_data[0])
print(targets)
y_test1 = np.eye(32)[targets]

# Restore the targets to the previous variables
Y_train_data = y_train1
y_test_data = y_test1

In [ ]:
print("X train length: ", len(X_train_data))
print("Y train length: ", len(Y_train_data))
print(Y_train_data[: 5])
print()
print("x test length: ", len(x_test_data))
print("y test length: ", len(y_train_data))
print(y_test_data[: 5])

## Categorize and Reshape the Data
* Without scalling the data

In [ ]:
# Reshape of Train Category
X_train_data = np.array([item[0] for item in train_data]).reshape(-1, 96, 96, 1)
Y_train_data = np.array([item[1] for item in train_data])

targets = np.array(Y_train_data).reshape(-1)
print(X_train_data[0])
print(targets)
y_train1 = np.eye(32)[targets]
print()
print("X_train length: ", len(X_train_data), "Y_train length: ", len(Y_train_data))
print()

# Reshape of Test Category
x_test_data = np.array([item[0] for item in test_data]).reshape(-1, 96, 96, 1)
y_test_data = np.array([item[1] for item in test_data])

targets = np.array(y_test_data).reshape(-1)
print(x_test_data[0])
print(targets)
y_test1 = np.eye(32)[targets]

print()
print("x_test length: ", len(x_test_data), "y_test length: ", len(y_test_data))

# Restore the targets to the previous variables
Y_train_data = y_train1
y_test_data = y_test1

## 2- Create the CNN Model

In [ ]:
# Initializing the cnn object.
cnn = input_data(shape = [None, 96, 96, 1], name='input')

cnn = conv_2d(cnn, 8, 8, activation='relu')
cnn = max_pool_2d(cnn, 8)

cnn = conv_2d(cnn, 16, 8, activation='relu')
cnn = max_pool_2d(cnn, 8)

cnn = conv_2d(cnn, 32, 8, activation='relu')
cnn = max_pool_2d(cnn, 8)

cnn = conv_2d(cnn, 64, 8, activation='relu')
cnn = max_pool_2d(cnn, 8)

cnn = conv_2d(cnn, 128, 8, activation='relu')
cnn = max_pool_2d(cnn, 8)

cnn = fully_connected(cnn, 64, activation='relu')
cnn = dropout(cnn, 0.30) # Avoid 30% of data overfitting in the model.

cnn = fully_connected(cnn, 32, activation='softmax')
cnn = regression(cnn, optimizer='adam', learning_rate=0.01, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(cnn, tensorboard_dir='AFG_logs')
model.fit(
    {'input': X_train_data},
    {'targets': Y_train_data},
    n_epoch=15,
    validation_set=({'input': x_test_data}, {'targets': y_test_data}),
    snapshot_step=500,
    show_metric=True,
    run_id="AFG_model_01.model"
)

model.save("AFG_model_01.model")
score = model.evaluate(x_test_data_data, y_test_data)
print('Test accuarcy: %0.4f%%' %(score[0] * 100))